<a href="https://colab.research.google.com/github/RicoStaedeli/NLP2025_CQG/blob/main/4_Finetuned_Generation_schema.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuned Predictions
In this file we generate the finetuned predictions

## Setup

In [ ]:
!pip install unsloth_zoo
!pip install --no-deps unsloth
!pip install -U transformers
!pip install accelerate bitsandbytes
!pip install -U peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.1/138.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/363.4 MB 198.3 MB/s eta 0:00:02

In [ ]:
import torch
from google.colab import userdata
import logging
import transformers
import os
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer
from bitsandbytes import nn as bnb_nn
from unsloth import FastLanguageModel
from tqdm.auto import tqdm
import pandas as pd
import json
from collections import defaultdict
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
token = userdata.get('GITHUB')
repo_url = f"https://{token}@github.com/RicoStaedeli/NLP2025_CQG.git"

!git clone {repo_url}

In [ ]:
################################################################################
#######################   PATH VARIABLES        ################################
################################################################################

model_name = "Meta-Llama-3.1-8B-Instruct_SFT_2_DPO_schema_prompt"
model_id = "ricostaedeli/Meta-Llama-3.1-8B-Instruct_SFT_2_DPO" #"meta-llama/Llama-3.1-8B-Instruct" # ricostaedeli/Meta-Llama-3.1-8B-Instruct_DPO

test_dataset_path = f"/content/NLP2025_CQG/Data/Processed/test.csv"

results_path = os.path.join(os.getcwd(), f"/content/NLP2025_CQG/Evaluation/Results/results_{model_name}.json")

log_base_path = f"/content/NLP2025_CQG/Logs/"
os.makedirs(log_base_path, exist_ok=True)

log_path = log_base_path + f"4_cqs_generation_{model_name}.log"


################################################################################
#######################   STATIC VARIABLES      ################################
################################################################################


In [ ]:
# Setup logger manually
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create file handler (only if not already added)
if not logger.handlers:
    fh = logging.FileHandler(log_path)
    fh.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    logger.addHandler(fh)

# Detect device
device = torch.device(
    "mps" if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available()
    else "cpu"
)

# Log the device info
logger.info("--------  Start with Baseline Generation  -------------")
logger.info(f'Device selected: {device}')
logger.info(f'Results Path: {results_path}')
logger.info(f'Log Path: {log_path}')
logger.info("--------------------------------------------------------")

## Generate Answers

Preprocess dataset

In [ ]:
df = pd.read_csv(test_dataset_path)

# Define the schema types
schemas = {
    "CauseToEffect": """Examples are:
    What if the job is one that will cause their illness to manifest and affect their job performance?
    If so, do you think the lack of any of the pieces influence the decisions they make about whether to create the digital widget or which digital widget to create?
    If miners and the working class are such a small percentage of the population as to not make a difference in a straight up popular vote, then why do we let them have so much influence?
    """,

    "ExpertOpinion": """'Examples are:
    Do you have a scientific source that confirm that women only wants one partner to a greater degree than men?
    If you are just stating facts, the surely you can point to some studies that show most women are narcissistic sociopaths?
    Where in any scientific textbook or journal have you seen evidence of sex being described as a spectrum?
    """,

    "Analogy": """Examples are:
    If so, would these groups resemble what are commonly referred to as races?
    If whites are guilty of enjoying the benefits of stolen land after being here for two generations, why would that not mean a 2nd generation American POC not have a similar benefit?
    But why is the way her hair naturally grows from her head seen as less professional than chemically altering to mimic straight white European hair?
    """,

    "FearAppeal": """'Examples are:
    If he did such a great job making peace, why do we need sanctions to pressure North Korea into stopping its violent threats?
    And if we should strive to stop all killings then why would the implement of this murder be spared if it causes a very large number of deaths but was intended to cause none?
    If you say you can, then are you implying that you see Islam as being no worse?
    """
}

# Prepare the JSON structure
json_data = []
for idx, row in df.iterrows():
    context = row['input'].strip()
    original_id = row['id']

    for schema_key, schema_value in schemas.items():
        entry = {
            "prompt": [
                {
                    "role": "system",
                    "content": "You generate concise, critical, single-sentence questions for argumentative contexts, matching specified question schemas."
                },
                 {
                    "role": "user",
                    "content": f"Generate one critical question addressing the provided context. Ensure it matches the schema:{schema_key}.\n\nSchema definition:\n{schema_value}\n\nContext: {context} Respond only with the question and nothing else."
                },
                 {
                    "role": "assistant",
                    "content": "Question: "
                }

            ],
            "id": f"{original_id}",
            "schema" : schema_key,
            "input" : context

        }
        json_data.append(entry)


# Save the JSON data to a file
temp_output_file = 'processed_dataset.json'
with open(temp_output_file, 'w') as f:
    json.dump(json_data, f, indent=4)

## Generation with transformers

In [ ]:
# del model
# gc.collect()
# torch.cuda.empty_cache()

Load pipeline with quantized model for less GPU usage


In [ ]:
'''
pipeline = transformers.pipeline(
    "text-generation",
    model = model_id,
    model_kwargs={"torch_dtype": torch.float16},
)
'''

In [ ]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {
            "load_in_4bit": True,
            "bnb_4bit_compute_dtype": torch.float16,
        },
        "low_cpu_mem_usage": True,
    },
    device_map="auto",
)

### Single generation

In [ ]:
'''
def get_response(query, max_tokens=512, temperature=0.6, top_p=0.9):

    prompt = pipeline.tokenizer.apply_chat_template(
        query, tokenize=False, add_generation_prompt=True
    )

    outputs = pipeline(
        prompt,
        max_new_tokens=max_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
    )
    response = outputs[0]["generated_text"][len(prompt):]
    return response
    '''

In [ ]:
'''
with open(temp_output_file, "r") as f:
    data = json.load(f)

combined_output = {}
i = 0
for item in data:
    print(f"Number: {i}")
    item_id = item["id"]
    schema = item["schema"]
    message = item["prompt"]
    input = item["input"]
    question = get_response(message).strip()

    if item_id not in combined_output:
        combined_output[item_id] = {"input": input, "cqs": []}

    combined_output[item_id]["cqs"].append({"schema": schema, "cq": question})
    i = i + 1
    if i == 2:
      break


# Save the combined questions
with open(results_path, "w") as f:
    json.dump(combined_output, f, indent=4)
print(f"Combined questions saved to {results_path}")
'''

### Batch processing

In [ ]:
def get_responses(queries, max_tokens=512, temperature=0.6, top_p=0.9):
    # Prepare the prompts
    prompts = [pipeline.tokenizer.apply_chat_template(q, tokenize=False, add_generation_prompt=True) for q in queries]

    # Run generation in batches to avoid OOM
    all_responses = []
    batch_size = 250
    for i in range(0, len(prompts), batch_size):
        batch_prompts = prompts[i:i+batch_size]

        # Run the pipeline in batch mode
        outputs = pipeline(
            batch_prompts,
            max_new_tokens=max_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
        )

        # Extract responses
        for prompt, output_list in zip(batch_prompts, outputs):
            # Each output_list is a list with a single dictionary
            generated_text = output_list[0]["generated_text"]
            response = generated_text[len(prompt):].strip()
            all_responses.append(response)
            print(response)

    return all_responses

# Load the data
with open(temp_output_file, "r") as f:
    data = json.load(f)

queries = [item["prompt"] for item in data]
responses = get_responses(queries)

# Combine the outputs
combined_output = {}
for item, response in zip(data, responses):
    item_id = item["id"]
    schema = item["schema"]
    input = item["input"]

    if item_id not in combined_output:
        combined_output[item_id] = {"input": input, "cqs": []}

    combined_output[item_id]["cqs"].append({"schema": schema, "cq": response})

# Save the combined questions
with open(results_path, "w") as f:
    json.dump(combined_output, f, indent=4)

print(f"Combined questions saved to {results_path}")

In [ ]:
try:
    os.remove(temp_output_file)
    print(f"File '{temp_output_file}' deleted successfully.")
except FileNotFoundError:
    print(f"File '{temp_output_file}' not found.")
except PermissionError:
    print(f"Permission denied to delete '{temp_output_file}'.")
except Exception as e:
    print(f"Error deleting file: {e}")

## Commit & Push

In [ ]:
os.chdir("NLP2025_CQG")
!ls

In [ ]:
!git config --global user.name "Rico Städeli"
!git config --global user.email "rico@yabrriga.ch"


commit_message = f"Finetuned generation"
!git add .
!git commit -m "{commit_message}"
!git push